In [1]:
import import_ipynb
import matplotlib.pyplot as plt
from FULL_DATA import final_df
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords

importing Jupyter notebook from FULL_DATA.ipynb
importing Jupyter notebook from Create_Labels.ipynb
(6380, 4)
4400
4467
4377
(4400, 7)
(4377, 7)
(4437, 7)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\grays\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# threshold = .1
# final_df['NEXT_RATING'] = final_df.groupby('TICKER')['ratings'].shift(1)
# final_df['Rating_Change'] = (final_df['ratings']-final_df['NEXT_RATING']) 
# # final_df['%_Delta'] = (final_df['Rating_Change']/final_df['NEXT_RATING']) 
# final_df.dropna(inplace = True)
# print(final_df.shape[0])
# final_df['SENTIMENT'] = [1 if x >= threshold else -1 if  x<=-threshold else 0 for x in final_df['Rating_Change']]
# final_df.reset_index(inplace = True)


# final_df['NEXT_RATING'] = final_df.groupby('TICKER')['ratings'].shift(1)
# final_df['Rating_Change'] = (final_df['ratings']-final_df['NEXT_RATING']) 
# final_df.dropna(inplace = True)
# print(final_df.shape[0])
# final_df['SENTIMENT'] = [1 if x > 0 else 0 if x ==0 else 2 for x in final_df['Rating_Change']]
# final_df.head(20)



final_df['SENTIMENT'] = [0 if (x > 1 and x<3.4999) else 1 if (x > 3.5 and x<4.49999) else -1  for x in final_df['ratings']]
print(final_df['SENTIMENT'].value_counts())
print(final_df.shape)
final_df.head()


 1    2367
 0    1144
-1     866
Name: SENTIMENT, dtype: int64
(4377, 8)


,TRANSCRIPTS,TICKER,TRANSCRIPT_DATE,RATINGS_DATE,ratings,DATE_DIFF,KEY,SENTIMENT
14598,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-07-31,2018-09-28,4.714,59 days,A_2018-09-28,-1
14595,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-11-19,2018-12-31,4.571,42 days,A_2018-12-31,-1
14588,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-01-31,2019-03-29,5.000,57 days,A_2019-03-29,-1
14585,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-05-14,2019-06-28,4.867,45 days,A_2019-06-28,-1
14582,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-08-14,2019-09-30,4.286,47 days,A_2019-09-30,1


In [ ]:
# all_words = []
# # nltk.download('wordnet')
# from nltk.stem import PorterStemmer 
# ps = PorterStemmer()
# from nltk.stem import WordNetLemmatizer 
# lemmatizer = WordNetLemmatizer() 
# counter = 0
# for i in final_df['TRANSCRIPTS']: 
#     print("counter",counter)
#     counter+=1
#     for j in word_tokenize(i.lower()):
#         lem_word = lemmatizer.lemmatize(j)
#         stem_word = ps.stem(lem_word)
#         if stem_word in stopwords.words('english'):
#             pass
#         else:
#             all_words.append(stem_word)
# all_words            

In [ ]:
final_df['SENTIMENT'].value_counts()
# len(all_words)

In [ ]:
# iterate through each sentence in the file 
data = [] 
for i in final_df['TRANSCRIPTS']: 
    temp = [] 
    # tokenize the sentence into words 
#     print(i)
    for j in word_tokenize(i): 
        if j in temp:
            pass
        elif j in stopwords.words('english'):
            pass
        else:
            temp.append(j.lower()) 
    data.append(temp)         
data

In [ ]:
tf=TfidfVectorizer(lowercase=True,max_df = .9,min_df=.1,ngram_range = (1,1))
text_tf= tf.fit_transform(final_df['TRANSCRIPTS'])
tfidf = dict(zip(tf.get_feature_names(), tf.idf_))
# tfidf

In [ ]:
# processed_article = article_text.lower()
# processed_article = re.sub('[^a-zA-Z]', ' ', processed_article )
# processed_article = re.sub(r'\s+', ' ', processed_article)
# final_df['TRANSCRIPTS'] = final_df['TRANSCRIPTS'].apply(word_tokenize)

max_len=20
from gensim.models import Word2Vec
word2vec = Word2Vec(data, min_count=2,size = max_len, window = 5)
vocabulary = word2vec.wv.vocab
vocabulary

In [ ]:
avg_list = []   
import numpy as np
for i in final_df['TRANSCRIPTS']: 
    vec = np.zeros(max_len).reshape((1, max_len))
    count = 0
#     print("iiiiiiiiiiiiiiiiiiiiii",i)
    for j in word_tokenize(i):
#         print(j)
        try:
            vec += word2vec[j].reshape((1, max_len)) * tfidf[j]
            count += 1.
        except KeyError: 
            continue
    if count != 0:
        vec /= count
    avg_list.append(vec[-1])

In [ ]:
# import pandas as pd

# print(avg_list[0:2])
pd.DataFrame(avg_list)

In [ ]:
# word2vec.similarity('computer', 'phone')
word2vec.wv.most_similar('shares')

In [ ]:
import pandas as pd
# X_train, X_test, y_train, y_test = train_test_split(
#     pd.DataFrame(avg_list), final_df['SENTIMENT'], test_size=0.15, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(
    pd.DataFrame(avg_list), final_df['Rating_Change'], test_size=0.15, random_state=1)


In [ ]:
# X_train
y_train

In [ ]:
clf = LogisticRegression(random_state=1, multi_class= 'multinomial',solver = 'lbfgs',max_iter=200).fit(X_train, y_train)
print("Train Logistic Accuracy:",metrics.accuracy_score(y_train, clf.predict(X_train)))
print("Test Logistic Accuracy:",metrics.accuracy_score(y_test, clf.predict(X_test)))

In [ ]:
clf = RandomForestClassifier(max_depth=6, random_state=1,n_estimators=10000)
clf.fit(X_train, y_train)
print("Train RF Accuracy:",metrics.accuracy_score(y_train, clf.predict(X_train)))
print("Test RF Accuracy:",metrics.accuracy_score(y_test, clf.predict(X_test)))
# clf.predict_proba(X_test)
print("Confusion Matrix RF:",metrics.multilabel_confusion_matrix(y_test, clf.predict(X_test)))

In [ ]:
import tensorflow 
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import SGD
epochs = 20000
lr =.1
model = Sequential()
model.add(Dense(100, activation='sigmoid', input_dim=max_len))
model.add(Dropout(0.1))
model.add(Dense(100, activation='relu'))
model.add(Dense(1))
model.add(Activation('linear'))
sgd = SGD(lr=0.05, momentum=0.9, nesterov=True)
model.compile(loss='mae', optimizer=sgd)
# early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=.0001, patience=50, verbose=2, mode='auto', baseline=None, restore_best_weights=True)
model.fit(np.array(X_train), np.array(y_train), epochs=epochs, batch_size=50, verbose=2,validation_data=(np.array(X_test), np.array(y_test)), shuffle=False)

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=128, verbose=2)
print(score)